libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import time
import chess
import chess.pgn
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

chess data import

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/113.0'
}

players = {
    "Magnus Carlsen": "https://www.chess.com/games/search?fromSearchShort=1&p1=Magnus%20Carlsen&playerId=822231&page=",
    "Hikaru Nakamura": "https://www.chess.com/games/search?fromSearchShort=1&p1=Hikaru%20Nakamura&playerId=291573&page=",
    "Garry Kasparov": "https://www.chess.com/games/search?fromSearchShort=1&p1=Garry%20Kasparov&playerId=21779&page=",
    "Viswanathan Anand": "https://www.chess.com/games/search?fromSearchShort=1&p1=Viswanathan%20Anand&playerId=284082&page=",
    "Fabiano Caruana": "https://www.chess.com/games/search?fromSearchShort=1&p1=Fabiano%20Caruana&playerId=42382&page=",
    "Alexander Morozevich": "https://www.chess.com/games/search?fromSearchShort=1&p1=Alexander%20Morozevich&playerId=32913&page=",
    "Alan Pichot": "https://www.chess.com/games/search?fromSearchShort=1&p1=Alan%20Pichot&playerId=813915&page=",
    "Bobby Fischer": "https://www.chess.com/games/search?fromSearchShort=1&p1=Bobby%20Fischer&playerId=1092583&page=",
    "Aleksandar Indjic": "https://www.chess.com/games/search?fromSearchShort=1&p1=Aleksandar%20Indjic&playerId=823770&page=",
    "Alexander Donchenko": "https://www.chess.com/games/search?fromSearchShort=1&p1=Alexander%20Donchenko&playerId=285507&page=",
    "Aleksey Dreev": "https://www.chess.com/games/search?fromSearchShort=1&p1=Aleksey%20Dreev&playerId=28661&page=",
    "David Paravyan": "https://www.chess.com/games/search?fromSearchShort=1&p1=David%20Paravyan&playerId=827967&page=",
    "Ding Liren": "https://www.chess.com/games/search?fromSearchShort=1&p1=Ding%20Liren&playerId=42388&page=",
    "Francisco Vallejo Pons": "https://www.chess.com/games/search?fromSearchShort=1&p1=Francisco%20Vallejo%20Pons&playerId=34529&page=",
    "George Meier": "https://www.chess.com/games/search?fromSearchShort=1&p1=Georg%20Meier&playerId=40740&page=",
    "Gregory Kaidanov": "https://www.chess.com/games/search?fromSearchShort=1&p1=Gregory%20Kaidanov&playerId=19769&page=",
    "Ivan Saric": "https://www.chess.com/games/search?fromSearchShort=1&p1=Ivan%20Saric&playerId=43211&page=",
    "Jan-Krzysztof Duda": "https://www.chess.com/games/search?fromSearchShort=1&p1=Jan-Krzysztof%20Duda&playerId=822255&page=",
    "Ju Wenjun": "https://www.chess.com/games/search?fromSearchShort=1&p1=Ju%20Wenjun&playerId=42865&page=",
    "Kiril Georgiev": "https://www.chess.com/games/search?fromSearchShort=1&p1=Kiril%20Georgiev&playerId=26897&page=",
    "Le Quang Liem": "https://www.chess.com/games/search?fromSearchShort=1&p1=Le%20Quang%20Liem&playerId=41329&page=",
    "Maxim Matlakov": "https://www.chess.com/games/search?fromSearchShort=1&p1=Maxim%20Matlakov&playerId=41788&page=",
    "Pavel Ponkratov": "https://www.chess.com/games/search?fromSearchShort=1&p1=Pavel%20Ponkratov&playerId=42016&page=",
    "Richard Rapport": "https://www.chess.com/games/search?fromSearchShort=1&p1=Richard%20Rapport&playerId=290480&page=",
    "Sergey Karjakin": "https://www.chess.com/games/search?fromSearchShort=1&p1=Sergey%20Karjakin&playerId=39821&page=",
    "Sergei Zhigalko": "https://www.chess.com/games/search?fromSearchShort=1&p1=Sergei%20Zhigalko&playerId=40749&page=",
    "Yu Yangyi": "https://www.chess.com/games/search?fromSearchShort=1&p1=Yu%20Yangyi&playerId=287192&page=",
    "Romain Edouard": "https://www.chess.com/games/search?fromSearchShort=1&p1=Romain%20Edouard&playerId=43361&page=",
    "Saleh Salem": "https://www.chess.com/games/search?fromSearchShort=1&p1=Saleh%20Salem&playerId=1034323&page=",
    "Sanan Sjugirov": "https://www.chess.com/games/search?fromSearchShort=1&p1=Sanan%20Sjugirov&playerId=43882&page="
}

pgn_file = "../data/all_games.pgn"  # Path to save the combined PGN file

for player, player_url in players.items():
    game_ids = []

    for page in tqdm(range(40)):   # this attempts to grab the first 25 pages
        URL = f'{player_url}{page}'
        response = requests.get(URL, headers=headers)
        if response.status_code != 200:
            print(f'Error retrieving games for {player} on page {page}')
            break
        soup = BeautifulSoup(response.text)
        page_game_ids = [x.get('href') for x in soup.find_all('a', {'class': 'master-games-clickable-link master-games-td-user'})]
        page_game_ids = [x.split('/')[-1] for x in page_game_ids]
        game_ids.extend(page_game_ids)
        time.sleep(15)

    # Join game ids
    game_ids = ','.join(game_ids)

    DOWNLOAD_URL = f'https://www.chess.com/games/downloadPgn?game_ids={game_ids}'
    download_response = requests.get(DOWNLOAD_URL)

    with open(pgn_file, 'ab') as file:
        file.write(download_response.content)

formatting / processing data

In [ ]:
import pandas as pd
import numpy as np


def extract_moves(game):
    moves = []
    node = game
    move_number = 1
    board = chess.Board()  # Create a new board object
    while not node.is_end():
        node = node.variations[0]
        move = node.move
        color = "Black" if node.board().turn == chess.WHITE else "White"  # Reverse the color assignment
        move_from = chess.square_name(move.from_square)
        move_to = chess.square_name(move.to_square)

        move_combined = move_from + move_to  # Combine move squares into one string

        moves.append((color, move_number, move_combined, get_piece_from_move(board, move)))
        move_number += 1
        board.push(move)  # Push the move onto the board

    return moves

def get_piece_from_move(board, move):
    piece = board.piece_at(move.from_square)
    if piece is None:
        return "Pawn"
    elif piece.piece_type == chess.KNIGHT:
        return "Knight"
    elif piece.piece_type == chess.BISHOP:
        return "Bishop"
    elif piece.piece_type == chess.ROOK:
        return "Rook"
    elif piece.piece_type == chess.QUEEN:
        return "Queen"
    elif piece.piece_type == chess.KING:
        return "King"
    else:
        return "Pawn"

# Initialize empty lists for each column of the DataFrame
game_id_list = []
white_result_list = []
black_result_list = []
color_list = []
move_number_list = []
move_list = []
piece_list = []

pgn_file = "../data/all_games.pgn"  # Path to the combined PGN file

# Open the PGN file
with open(pgn_file) as f:
    game_number = 1
    while True:
        game = chess.pgn.read_game(f)
        if game is None:
            break

        # Extract information about the game
        headers = game.headers
        game_id = game_number  # Use the game_number as the game ID
        game_number += 1
        result = headers["Result"]
        white_result = 0
        black_result = 0

        if result == "1-0":
            white_result = 1
            black_result = 0
        elif result == "0-1":
            white_result = 0
            black_result = 1

        # Extract the moves
        moves = extract_moves(game)

        # Append the information for the game to the lists
        for color, move_number, move_combined, piece in moves:
            game_id_list.append(game_id)
            white_result_list.append(white_result)
            black_result_list.append(black_result)
            color_list.append(color)
            move_number_list.append(move_number)
            move_list.append(move_combined)
            piece_list.append(piece)

# Create a DataFrame from the extracted data
data = {
    "Game ID": game_id_list,
    "White Result": white_result_list,
    "Black Result": black_result_list,
    "Color": color_list,
    "Move Number": move_number_list,
    "Move": move_list,
    "Piece": piece_list
}
df = pd.DataFrame(data)
df

read in 30k games data

In [2]:
df = pd.read_csv('df.csv')
df

,Game ID,White Result,Black Result,Color,Move Number,Move,Piece
0,1,1,0,White,1,a2a4,Pawn
1,1,1,0,Black,2,g8f6,Knight
2,1,1,0,White,3,d2d4,Pawn
3,1,1,0,Black,4,d7d5,Pawn
4,1,1,0,White,5,g1f3,Knight
...,...,...,...,...,...,...,...
2529402,29319,1,0,White,139,a7d4,Queen
2529403,29319,1,0,Black,140,f6f7,King
2529404,29319,1,0,White,141,a6a7,Pawn
2529405,29319,1,0,Black,142,c6c5,Pawn


encode dataframe

In [3]:
# Perform one-hot encoding on the 'Move', 'Piece', and 'Color' columns
df_encoded = pd.get_dummies(df, columns=['Move', 'Piece', 'Color'])

# Verify the encoded DataFrame
print(df_encoded.head())

   Game ID  White Result  Black Result  Move Number  Move_a1a1  Move_a1a2  \
0        1             1             0            1          0          0   
1        1             1             0            2          0          0   
2        1             1             0            3          0          0   
3        1             1             0            4          0          0   
4        1             1             0            5          0          0   

   Move_a1a3  Move_a1a4  Move_a1a5  Move_a1a6  ...  Move_h8h6  Move_h8h7  \
0          0          0          0          0  ...          0          0   
1          0          0          0          0  ...          0          0   
2          0          0          0          0  ...          0          0   
3          0          0          0          0  ...          0          0   
4          0          0          0          0  ...          0          0   

   Piece_Bishop  Piece_King  Piece_Knight  Piece_Pawn  Piece_Queen  \
0         

create board states array using encoded df

In [5]:
# Define a mapping between piece names and piece symbols
piece_mapping = {
    'Pawn': chess.PAWN,
    'Knight': chess.KNIGHT,
    'Bishop': chess.BISHOP,
    'Rook': chess.ROOK,
    'Queen': chess.QUEEN,
    'King': chess.KING
}

# Initialize an empty list for board states
board_states = []

# Group the data by 'Game ID'
grouped_data = df_encoded.groupby('Game ID')

# Iterate over each group
for group_name, group_data in grouped_data:
    # Create a new board object for each game
    board = chess.Board()
    
    # Initialize an empty list for the current game's board state
    game_board_state = []
    
    # Iterate over the rows in the current group
    for _, row in group_data.iterrows():
        # Get the move and piece from the encoded columns
        move_columns = row[row.index.str.startswith('Move_')]
        move_uci = move_columns.idxmax()[5:] if not move_columns.isnull().all() else None
        
        # Check if the move is a null move
        if move_uci is not None and move_uci == '0000':
            continue
        
        piece_columns = row[row.index.str.startswith('Piece_')]
        piece = piece_columns.idxmax()[6:] if not piece_columns.isnull().all() else None
        
        try:
            # Try to create a Move object from the UCI notation
            move = chess.Move.from_uci(move_uci) if move_uci is not None else None
        except chess.InvalidMoveError:
            # If an InvalidMoveError occurs, skip this move and continue with the next one
            continue
        
        # Check if the move is legal for the current board position
        if move is not None and board.is_legal(move):
            # Ensure that the piece being moved matches the one in the dataset
            if piece is None or piece_mapping[piece] == board.piece_at(move.from_square).piece_type:
                # Add the board state to the current game's board states
                game_board_state.append(board.copy())
                
                # Make the move on the board
                board.push(move)
    
    # Add the board states of the current game to the overall list
    board_states.append(game_board_state)

# Convert board states to a NumPy array
board_states = np.array(board_states, dtype=object)

In [ ]:
# Save board_states to a file
with open('all_board_states.pkl', 'wb') as file:
    pickle.dump(board_states, file)

In [1]:
import pickle

# Load the cache from the pickle file
with open("board_states.pkl", "rb") as file:
    board_states = pickle.load(file)

board_states

In [ ]:
board_states

use stockfish to find the best move for all board states then store them in a cache

In [6]:
from stockfish import Stockfish

stockfish = Stockfish(path="..\stockfish\stockfish-windows-2022-x86-64-avx2.exe")

In [ ]:
import pickle

# Convert board states to FEN strings
fen_list = [board.fen() for game in board_states for board in game]

# Process fen_list and populate the cache with best moves
for fen in fen_list:
    # Set the position in Stockfish
    stockfish.set_fen_position(fen)

    # Get the best move from Stockfish
    best_move = stockfish.get_best_move()

    # Add the best move to the cache
    cache[fen] = best_move

# Save the cache as a pickle file
with open("cache.pkl", "wb") as file:
    pickle.dump(cache, file)

In [10]:
from stockfish import Stockfish

# Initialize Stockfish engine
stockfish = Stockfish(path="..\stockfish\stockfish-windows-2022-x86-64-avx2.exe")

# Get the current parameters
parameters = stockfish.get_parameters()

# Print the parameters
for param, value in parameters.items():
    print(f"{param}: {value}")

Debug Log File: 
Contempt: 0
Min Split Depth: 0
Ponder: false
MultiPV: 1
Skill Level: 20
Move Overhead: 10
Minimum Thinking Time: 20
Slow Mover: 100
UCI_Chess960: false
UCI_LimitStrength: false
UCI_Elo: 1350
Threads: 1
Hash: 16


In [15]:
import pickle

with open('updated_cache.pkl', 'rb') as file:
    cache = pickle.load(file)

In [16]:
len(cache)

1185797

In [17]:
# Convert board states to FEN strings
fen_list = [board.fen() for game in board_states for board in game]

# Process fen_list and populate the cache with best moves
for fen in fen_list[len(cache):]:
    if fen not in cache:
        # Set the position in Stockfish
        stockfish.set_fen_position(fen)

        # Get the best move from Stockfish
        best_move = stockfish.get_best_move()

        # Add the best move to the cache
        cache[fen] = best_move

KeyboardInterrupt: 

In [11]:
cache2 = cache

In [18]:
len(cache)

1965350

In [14]:
# Save the cache as a pickle file
with open("updated_cache.pkl", "wb") as file:
    pickle.dump(cache, file)

In [ ]:
%%timeit 
fen in cache

In [ ]:
%%timeit

stockfish.set_fen_position(fen)

stockfish.get_best_move()

In [ ]:
len(cache)